In [ ]:
# Quantum Simulation Visualization Demo

"""
Demonstration of the visualization module capabilities.
This notebook shows how to create publication-quality plots for:
- Time evolution visualization
- Method comparison plots
- Absorption spectrum visualization
- Error analysis plots
- Performance benchmarking charts
- Comprehensive analysis reports
"""

import numpy as np
import matplotlib.pyplot as plt
from quantum_simulation import (
    create_quantum_system,
    create_exact_solver,
    create_trotter_solver,
    create_magnus_solver,
    create_state_analyzer,
    create_spectrum_analyzer,
    create_performance_analyzer,
    create_time_evolution_plotter,
    create_spectrum_plotter,
    create_error_analysis_plotter,
    create_performance_plotter,
    create_comprehensive_analysis_plots,
    PlotStyle,
    SimulationConfig
)

print("=== Quantum Simulation Visualization Demo ===")

# Setup configuration for visualization demo
viz_config = SimulationConfig(
    time_end=30.0,       # Good range for visualization
    num_time_points=15,  # Good resolution for plotting
    max_optimization_steps=50,  # Quick VQA for demo
    num_layers=2         # Simple ansatz
)

print(f"Visualization configuration:")
print(f"- Time range: {viz_config.time_start} to {viz_config.time_end}")
print(f"- Time points: {viz_config.num_time_points}")
print(f"- Molecule: {viz_config.atom_string}")

# Custom plot styling
custom_style = PlotStyle(
    figsize=(12, 8),
    dpi=150,  # Lower DPI for demo
    style='seaborn-v0_8',
    color_palette='Set2',
    font_size=12,
    line_width=2.5,
    marker_size=8.0
)

print(f"\nUsing custom plot style:")
print(f"- Figure size: {custom_style.figsize}")
print(f"- DPI: {custom_style.dpi}")
print(f"- Style: {custom_style.style}")
print(f"- Color palette: {custom_style.color_palette}")


In [ ]:
# Step 1: Generate Data for Visualization

print("\n=== Step 1: Generating Data ===")

# Create quantum system
system = create_quantum_system(viz_config)
ground_state, ground_energy = system.compute_ground_state_vqe()
time_points = viz_config.get_time_points()

print(f"System created:")
print(f"- Qubits: {system.static_hamiltonian.num_qubits}")
print(f"- Ground energy: {ground_energy:.6f} hartree")

# Run multiple solvers for comparison
print("\nRunning solvers...")
solvers_data = {}

# Exact solver
exact_solver = create_exact_solver(system, viz_config)
exact_states = exact_solver.evolve(time_points, ground_state)
solvers_data['exact'] = exact_states
print("✅ Exact solver completed")

# Trotter 1st order
trotter1_solver = create_trotter_solver(system, viz_config, order=1)
trotter1_states = trotter1_solver.evolve(time_points, ground_state)
solvers_data['trotter1'] = trotter1_states
print("✅ Trotter 1st order completed")

# Trotter 2nd order
trotter2_solver = create_trotter_solver(system, viz_config, order=2)
trotter2_states = trotter2_solver.evolve(time_points, ground_state)
solvers_data['trotter2'] = trotter2_states
print("✅ Trotter 2nd order completed")

# Magnus 2nd order
magnus_solver = create_magnus_solver(system, viz_config)
magnus_states = magnus_solver.evolve(time_points, ground_state)
solvers_data['magnus'] = magnus_states
print("✅ Magnus 2nd order completed")

print(f"\nData generation completed for {len(solvers_data)} methods")


In [ ]:
# Step 2: Time Evolution Plotting

print("\n=== Step 2: Time Evolution Visualization ===")

# Create analyzers and plotters
state_analyzer = create_state_analyzer(system, viz_config)
time_plotter = create_time_evolution_plotter(custom_style)

# Perform analysis
comparison_results = state_analyzer.compare_methods(solvers_data, reference_method='exact')

# Extract data for plotting
times = np.array(comparison_results.data['times'])
dipole_data = comparison_results.data['dipole_moments']
fidelity_data = comparison_results.data['fidelities']

print(f"Analysis completed:")
print(f"- Methods analyzed: {len(dipole_data)}")
print(f"- Time points: {len(times)}")

# Plot 1: Basic dipole evolution
print("\n📊 Creating dipole evolution plot...")
fig1 = time_plotter.plot_dipole_evolution(
    times, dipole_data,
    title="Dipole Moment Evolution - Method Comparison"
)
plt.show()

print("✅ Dipole evolution plot created")

# Plot 2: Dipole evolution with error analysis
print("\n📊 Creating dipole evolution with error analysis...")
fig2 = time_plotter.plot_dipole_comparison_with_error(
    times, dipole_data,
    reference_method='exact',
    title="Dipole Evolution with Error Analysis"
)
plt.show()

print("✅ Dipole evolution with error plot created")

# Plot 3: State fidelity evolution
print("\n📊 Creating state fidelity evolution plot...")
fig3 = time_plotter.plot_state_fidelity_evolution(
    times, fidelity_data,
    reference_method='exact',
    title="Quantum State Fidelity Evolution"
)
plt.show()

print("✅ State fidelity evolution plot created")

print("\n🎨 Time evolution visualization completed!")


In [ ]:
# Step 3: Spectrum Analysis Visualization

print("\n=== Step 3: Spectrum Analysis Visualization ===")

# Create spectrum analyzer and plotter
spectrum_analyzer = create_spectrum_analyzer(system, viz_config)
spectrum_plotter = create_spectrum_plotter(custom_style)

# Compute absorption spectra for all methods
print("Computing absorption spectra...")
spectra_results = {}

for method, dipole_vals in dipole_data.items():
    print(f"  Processing {method}...")
    try:
        spectrum_data = spectrum_analyzer.analyze_absorption_spectrum(
            times=times,
            dipole_moments=dipole_vals,
            damping_gamma=0.001,
            window_type='blackman',
            normalize=True
        )
        spectra_results[method] = spectrum_data
        print(f"    ✅ {method}: {len(spectrum_data['omega_ev'])} frequency points")
    except Exception as e:
        print(f"    ❌ {method}: Failed - {e}")

print(f"\nSpectrum analysis completed for {len(spectra_results)} methods")

# Plot 1: Basic absorption spectra
print("\n📊 Creating absorption spectra comparison...")
fig4 = spectrum_plotter.plot_absorption_spectra(
    spectra_results,
    title="Absorption Spectra Comparison",
    energy_range=(0, 2.0)  # Focus on low energy region
)
plt.show()

print("✅ Absorption spectra plot created")

# Plot 2: Detailed spectrum analysis
print("\n📊 Creating detailed spectrum analysis...")
fig5 = spectrum_plotter.plot_spectrum_comparison_detailed(
    spectra_results,
    title="Detailed Absorption Spectrum Analysis"
)
plt.show()

print("✅ Detailed spectrum analysis plot created")

# Display peak information
print("\n📈 Peak Analysis Summary:")
print("-" * 40)
for method, spectrum_data in spectra_results.items():
    omega_ev = spectrum_data['omega_ev']
    sigma = spectrum_data['absorption_spectrum']
    
    # Find peak
    peak_idx = np.argmax(sigma)
    peak_energy = omega_ev[peak_idx]
    peak_intensity = sigma[peak_idx]
    
    print(f"{method:10}: Peak at {peak_energy:.3f} eV (intensity: {peak_intensity:.3f})")

print("\n🌈 Spectrum visualization completed!")


In [ ]:
# Step 4: Error Analysis and Performance Visualization

print("\n=== Step 4: Error Analysis & Performance Visualization ===")

# Create error analysis and performance plotters
error_plotter = create_error_analysis_plotter(custom_style)
perf_plotter = create_performance_plotter(custom_style)

# Performance benchmarking
print("Running performance benchmarking...")
perf_analyzer = create_performance_analyzer()

# Benchmark each solver (simulate timing data)
benchmark_data = []
solver_funcs = {
    'Exact': (exact_solver.evolve, exact_solver),
    'Trotter1': (trotter1_solver.evolve, trotter1_solver),
    'Trotter2': (trotter2_solver.evolve, trotter2_solver), 
    'Magnus2': (magnus_solver.evolve, magnus_solver)
}

for name, (func, solver) in solver_funcs.items():
    print(f"  Benchmarking {name}...")
    benchmark_result = perf_analyzer.benchmark_solver(
        func, name, time_points, ground_state
    )
    benchmark_data.append(benchmark_result)

# Create performance DataFrame
import pandas as pd
perf_df = pd.DataFrame(benchmark_data)
print(f"✅ Performance benchmarking completed for {len(benchmark_data)} methods")

# Plot 1: Error metrics comparison
print("\n📊 Creating error metrics comparison...")
error_data = comparison_results.data.get('dipole_errors', {})
if error_data:
    fig6 = error_plotter.plot_error_metrics_comparison(
        error_data,
        title="Error Metrics Comparison (vs Exact)"
    )
    plt.show()
    print("✅ Error metrics comparison plot created")
else:
    print("⚠️ No error data available for plotting")

# Plot 2: Performance comparison
print("\n📊 Creating performance comparison...")
fig7 = perf_plotter.plot_performance_comparison(
    perf_df,
    title="Solver Performance Comparison"
)
plt.show()

print("✅ Performance comparison plot created")

# Plot 3: Accuracy vs Performance tradeoff
print("\n📊 Creating accuracy vs performance tradeoff...")
if error_data and not perf_df.empty:
    # Prepare data for tradeoff plot
    accuracy_dict = {method: errors['mae'] for method, errors in error_data.items()}
    performance_dict = dict(zip(perf_df['solver_name'].str.lower(), perf_df['wall_time']))
    
    fig8 = error_plotter.plot_accuracy_vs_performance(
        accuracy_dict, performance_dict,
        title="Accuracy vs Performance Tradeoff"
    )
    plt.show()
    print("✅ Accuracy vs performance tradeoff plot created")

# Performance summary
print("\n⚡ Performance Summary:")
print("-" * 50)
for _, row in perf_df.iterrows():
    solver = row['solver_name']
    time_s = row['wall_time']
    success = "✅" if row['success'] else "❌"
    speedup = row.get('speedup', 1.0)
    print(f"{solver:10}: {success} Time: {time_s:6.3f}s  Speedup: {speedup:5.2f}x")

print("\n📊 Error analysis and performance visualization completed!")


In [ ]:
# Step 5: Comprehensive Analysis and Report Generation

print("\n=== Step 5: Comprehensive Analysis Plots ===")

# Demonstrate the all-in-one plotting function
print("Generating comprehensive analysis plots...")

# Use the convenient function to create all plots at once
output_dir = "visualization_output"
try:
    import os
    os.makedirs(output_dir, exist_ok=True)
    print(f"📁 Created output directory: {output_dir}")
except:
    output_dir = None
    print("⚠️ Could not create output directory, plots will not be saved")

# Generate comprehensive plots
all_figures = create_comprehensive_analysis_plots(
    analysis_results=comparison_results,
    spectrum_data=spectra_results,
    performance_df=perf_df,
    output_dir=output_dir,
    style=custom_style
)

print(f"\n🎨 Generated {len(all_figures)} comprehensive plots:")
for plot_name, figure in all_figures.items():
    print(f"  ✅ {plot_name}")

# Display final summary
print("\n" + "="*60)
print("           VISUALIZATION DEMO SUMMARY")
print("="*60)

print(f"\n📊 PLOTS CREATED:")
plot_descriptions = {
    'dipole_evolution': 'Time evolution of dipole moments',
    'dipole_with_error': 'Dipole evolution with error analysis',
    'fidelity_evolution': 'Quantum state fidelity over time',
    'absorption_spectra': 'Absorption spectra comparison',
    'spectrum_detailed': 'Detailed spectrum analysis with zoom',
    'error_metrics': 'Error metrics comparison (MAE, RMSE, Max)',
    'performance': 'Solver performance benchmarking',
    'accuracy_vs_performance': 'Accuracy vs performance tradeoff'
}

for plot_name in all_figures.keys():
    description = plot_descriptions.get(plot_name, 'Analysis plot')
    print(f"   • {plot_name}: {description}")

print(f"\n🎯 VISUALIZATION CAPABILITIES DEMONSTRATED:")
print(f"   • Time evolution plotting with custom styling")
print(f"   • Multi-method comparison with error analysis")
print(f"   • Spectrum analysis with multiple zoom levels")
print(f"   • Performance benchmarking visualization")
print(f"   • Publication-quality figure generation")
print(f"   • Automated plot export and organization")

print(f"\n🎨 STYLING FEATURES:")
print(f"   • Custom color palettes for method differentiation")
print(f"   • Consistent line styles and markers")
print(f"   • Professional typography and layout")
print(f"   • Grid and annotation enhancements")
print(f"   • Multiple export formats support")

print(f"\n💾 OUTPUT:")
if output_dir:
    print(f"   • All plots saved to: {output_dir}/")
    print(f"   • High-resolution PNG format")
    print(f"   • Ready for publication use")
else:
    print(f"   • Plots displayed interactively")
    print(f"   • Can be saved manually if needed")

print(f"\n🚀 NEXT STEPS:")
print(f"   • Customize PlotStyle for specific requirements")
print(f"   • Integrate with automated analysis pipelines")
print(f"   • Export to additional formats (PDF, SVG, EPS)")
print(f"   • Create interactive plots with Plotly (if available)")

print("\n" + "="*60)
print("🎉 VISUALIZATION DEMO COMPLETED SUCCESSFULLY!")
print("📊 The refactored visualization module provides:")
print("   - Professional plotting capabilities")
print("   - Publication-quality figure generation")
print("   - Comprehensive analysis visualization")
print("   - Automated plot generation and export")
print("="*60)
